# US Institutions Financial Report

In [18]:
# May need to install pandas directly in the Jupyter notebook with the following commands
#! pip3 install --user pandas
#! pip3 install --user psycopg
import pandas as pd
import psycopg
import matplotlib as plt
from credentials import DBNAME, HOST, USERNAME, PASSWORD

In [2]:
# Parameters
YEAR = 2020


In [3]:
# Set the connection and grab the data from SQL

# Connect to the database using parameters from credentials.py
conn = psycopg.connect(
    dbname=DBNAME,
    host=HOST,
    user=USERNAME,
    password=PASSWORD)

# Create a cursor and use it to submit/execute a query:
cur = conn.cursor()

select_cmd = """
    SELECT * FROM institutions 
    WHERE extracted_year <= to_date(%s::text ,'YYYY-MM-DD');
"""

# Save to a dataframe
df = pd.DataFrame(cur.execute(select_cmd, (YEAR,)))

# Close the connection
cur.close()
conn.close()

# Data Summary

# Summary of Current College Tuition Rates

# Best and Worst Performing Institutions by Loan-Repayment Rates

# Tuition and Loan Repayment Rates over Time

# SAT Scores and College Admission Rate

# Faculty Salary and Revenue Tuition